# Hyperparameter Optimization

## Component to optimize

The configuration to optimize must compute a runnable component with state

In [ ]:
from automl.hp_opt.hp_optimization_pipeline import Component_to_opt_type

print(Component_to_opt_type)

In [ ]:
import math
from automl.basic_components.state_management import StatefulComponent
from automl.basic_components.exec_component import ExecComponent
from automl.core.input_management import InputSignature
from automl.component import requires_input_proccess


class ExampleSchemaToOptimize(ExecComponent, StatefulComponent):

    '''Simply generated a function: [(x + offset) ^ power ]'''

    parameters_signature = {
        "power" : InputSignature(description="The power <x + offset> is raised to", default_value=1),
        "offset" : InputSignature(description="", default_value=0),
        "inner_ratio" : InputSignature(description="", default_value=1),
        "outer_ratio" : InputSignature(description="", default_value=1),

        "X" : InputSignature(description="The input", mandatory=False)
    }

    exposed_values = {"last_output" : 0}

    def _proccess_input_internal(self):
        super()._proccess_input_internal()

        self.power = self.get_input_value("power")
        self.offset = self.get_input_value("offset")
        self.inner_ratio = self.get_input_value("inner_ratio")
        self.outer_ratio = self.get_input_value("outer_ratio")

        self.generated_funcion = self._generated_function()

        self.X = self.get_input_value("X")


    def _generated_function(self):

        def generated_function(X : float) -> float:

            return math.pow((X + self.offset) * self.inner_ratio, self.power) * self.outer_ratio
        
        return generated_function

    @requires_input_proccess
    def get_generated_function(self):

        return self.generated_funcion
    
    @requires_input_proccess
    def _algorithm(self):

        if self.X != None:
            self.values["last_output"] = self.generated_funcion(self.X)

        else:
            raise Exception("To run you must define an X")

In [ ]:
base_example_to_optimize = ExampleSchemaToOptimize({
    "offset" : -0.5,
    "power" : 2,
    "inner_ration" : 0.2,
    "outer_ratio" : 0.5
})

In [ ]:
# the element to optimize configuration dict
from automl.utils.json_utils.json_component_utils import json_string_of_component

base_example_to_optimize_json = json_string_of_component(base_example_to_optimize)

print(base_example_to_optimize_json)

## The evaluator

We can use a simple evaluator which uses as the result the last output value

In [ ]:
from automl.fundamentals.evaluators.value_evaluators import ValueEvaluator

evaluator_to_use = ValueEvaluator({"value_to_use" : "last_output"})

## Hyperparameter Suggestion

An hyperparameter suggestion is an object wich defines the range of values an hyperparameter can take and the localization of that hyperparameter in the configuration to optimize

In [ ]:
from automl.hp_opt.hyperparameter_suggestion import HyperparameterSuggestion

In [ ]:
example_hp_sug = HyperparameterSuggestion(
    name="x_value",
    hyperparameter_localizations=[ # the list of localizations were the value must be changed
        ( # a single localization
            [], # the component localization in the component tree
            "X" # the localization of the value to change
        )
    ],
    value_suggestion=("float", {"low" : -10.0, "high" : 10.0}))

## The Hyperparameter Optimization Pipeline

In [ ]:
from automl.hp_opt.hp_optimization_pipeline import HyperparameterOptimizationPipeline

hp_opt_pipeline = HyperparameterOptimizationPipeline(
    {   "artifact_relative_directory" : "exp",
        "base_directory" : "data\\hp_exps",
        "create_new_directory" : True,
        "configuration_string" : base_example_to_optimize_json,
        "n_trials" : 20,
        "evaluator_component" : evaluator_to_use,
        "hyperparameters_range_list" : [example_hp_sug],
        "start_with_given_values" : False
    }
)

print(f"Hp optimization pipeline generated in path: {hp_opt_pipeline.get_artifact_directory()}")

In [ ]:
hp_opt_pipeline.run()

## Seeing results

### True function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

def plot_generated_function(component : ExampleSchemaToOptimize, x_min=-10, x_max=10, n_points=400):
    """
    Plots the generated function from a component over a given X range.
    Skips invalid values (e.g., negative base with fractional power).
    """

    gen_func = component.get_generated_function()  # the generated function
    xs = np.linspace(x_min, x_max, n_points)

    ys = []
    valid_xs = []

    for x in xs:
        y = gen_func(x)
        if isinstance(y, float) or isinstance(y, int):
            valid_xs.append(x)
            ys.append(y)

    plt.figure(figsize=(8, 5))
    plt.plot(valid_xs, ys)
    plt.xlabel("X")
    plt.ylabel("f(X)")
    plt.title("Generated Function Plot")
    plt.grid(True)
    plt.show()

In [ ]:
plot_generated_function(base_example_to_optimize, -10, 10)

### Hyperparameter Optimization Results

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import get_hp_opt_results_logger

hyperparameter_optimization_results = get_hp_opt_results_logger(hp_opt_pipeline.get_artifact_directory())

print(f"Hyperparameter_optimization_results in path: {hyperparameter_optimization_results.get_artifact_directory()}")

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import get_hp_opt_optuna_study


optuna_study = get_hp_opt_optuna_study(hyperparameter_optimization_results)


In [ ]:
import optuna

optuna.visualization.plot_slice(optuna_study, params=["x_value"])


In [ ]:
fig = optuna.visualization.plot_optimization_history(optuna_study)
fig.show()